## 6.10 Homework

The goal of this homework is to create a tree-based regression model for prediction apartment prices (column `'price'`).

In this homework we'll again use the New York City Airbnb Open Data dataset - the same one we used in homework 2 and 3.

You can take it from [Kaggle](https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data?select=AB_NYC_2019.csv)
or download from [here](https://raw.githubusercontent.com/alexeygrigorev/datasets/master/AB_NYC_2019.csv)
if you don't want to sign up to Kaggle.

Let's load the data:

In [1]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [2]:
columns = [
    'neighbourhood_group', 'room_type', 'latitude', 'longitude',
    'minimum_nights', 'number_of_reviews','reviews_per_month',
    'calculated_host_listings_count', 'availability_365',
    'price'
]

df = pd.read_csv('AB_NYC_2019.csv', usecols=columns)
df.reviews_per_month = df.reviews_per_month.fillna(0)

* Apply the log tranform to `price`
* Do train/validation/test split with 60%/20%/20% distribution. 
* Use the `train_test_split` function and set the `random_state` parameter to 1

In [3]:
from sklearn.model_selection import train_test_split

trainX_full, testX = train_test_split(df, test_size=0.2, random_state=1)
trainX, valX = train_test_split(trainX_full, test_size=0.25, random_state=1)

trainy, trainX = trainX['price'], trainX.drop('price', axis=1)
valy, valX = valX['price'], valX.drop('price', axis=1)
testy, testX = testX['price'], testX.drop('price', axis=1)

trainy = np.log1p(trainy)
valy = np.log1p(valy)
testy = np.log1p(testy)

Now, use `DictVectorizer` to turn train and validation into matrices:

In [4]:
from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
trainX_encoded = dv.fit_transform(trainX.to_dict(orient='records'))
valX_encoded = dv.transform(valX.to_dict(orient='records'))

## Question 1

Let's train a decision tree regressor to predict the price variable. 

* Train a model with `max_depth=1`

In [5]:
from sklearn.tree import DecisionTreeRegressor, export_text

dt_reg = DecisionTreeRegressor(max_depth=1)
dt_reg.fit(trainX_encoded, trainy)

print(export_text(dt_reg, feature_names=dv.feature_names_))

|--- room_type=Entire home/apt <= 0.50
|   |--- value: [4.29]
|--- room_type=Entire home/apt >  0.50
|   |--- value: [5.15]



Which feature is used for splitting the data?

* `room_type`
* `neighbourhood_group`
* `number_of_reviews`
* `reviews_per_month`

**Answer:** `room_type`

## Question 2

Train a random forest model with these parameters:

* `n_estimators=10`
* `random_state=1`
* `n_jobs=-1`  (optional - to make training faster)

In [6]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

def random_forest_rmse(n_estimators=10, max_depth=None):
    rf_reg = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=1, n_jobs=-1)
    rf_reg.fit(trainX_encoded, trainy)
    predictions = rf_reg.predict(valX_encoded)

    return np.sqrt(mean_squared_error(predictions, valy)).round(3)

random_forest_rmse(10)

0.462

What's the RMSE of this model on validation?

* 0.059
* 0.259
* 0.459
* 0.659

## Question 3

Now let's experiment with the `n_estimators` parameter

* Try different values of this parameter from 10 to 200 with step 10
* Set `random_state` to `1`
* Evaluate the model on the validation dataset

In [7]:
for n in range(10, 201, 10):
    print(f'{n:-3d}: {random_forest_rmse(n)}')

 10: 0.462
 20: 0.448
 30: 0.446
 40: 0.444
 50: 0.442
 60: 0.442
 70: 0.441
 80: 0.441
 90: 0.441
100: 0.44
110: 0.439
120: 0.439
130: 0.439
140: 0.439
150: 0.439
160: 0.439
170: 0.439
180: 0.439
190: 0.439
200: 0.439


After which value of `n_estimators` does RMSE stop improving?

- 10
- 50
- 70
- 120

## Question 4

Let's select the best `max_depth`:

* Try different values of `max_depth`: `[10, 15, 20, 25]`
* For each of these values, try different values of `n_estimators` from 10 till 200 (with step 10)
* Fix the random seed: `random_state=1`

In [8]:
{max_depth: min([random_forest_rmse(n, max_depth) for n in range(10, 201, 10)])
         for max_depth in [10, 15, 20, 25]}

{10: 0.44, 15: 0.436, 20: 0.438, 25: 0.439}

What's the best `max_depth`:

* 10
* 15
* 20
* 25

Bonus question (not graded):

Will the answer be different if we change the seed for the model?

## Question 5

We can extract feature importance information from tree-based models. 

At each step of the decision tree learning algorith, it finds the best split. 
When doing
 it, we can calculate "gain" - the reduction in impurity before and after the split. 
This gain is quite useful in understanding what are the imporatant features 
for tree-based models.

In Scikit-Learn, tree-based models contain this information in the `feature_importances_` field. 

For this homework question, we'll find the most important feature:

* Train the model with these parametes:
    * `n_estimators=10`,
    * `max_depth=20`,
    * `random_state=1`,
    * `n_jobs=-1` (optional)
* Get the feature importance information from this model

In [9]:
rf_reg2 = RandomForestRegressor(n_estimators=10, max_depth=20, random_state=1)
rf_reg2.fit(trainX_encoded, trainy)

for x in sorted(zip(dv.feature_names_, rf_reg2.feature_importances_), key=lambda x: x[1], reverse=True):
    print(f'{x[0]:>35s}: {x[1]:.3f}')

          room_type=Entire home/apt: 0.392
                          longitude: 0.154
                           latitude: 0.153
                   availability_365: 0.076
                  reviews_per_month: 0.054
                     minimum_nights: 0.053
                  number_of_reviews: 0.042
      neighbourhood_group=Manhattan: 0.034
     calculated_host_listings_count: 0.030
              room_type=Shared room: 0.005
             room_type=Private room: 0.004
         neighbourhood_group=Queens: 0.001
       neighbourhood_group=Brooklyn: 0.001
          neighbourhood_group=Bronx: 0.000
  neighbourhood_group=Staten Island: 0.000


What's the most important feature? 

* `neighbourhood_group=Manhattan`
* `room_type=Entire home/apt`	
* `longitude`
* `latitude`

**Answer:** `room_type=Entire home/apt`

## Question 6

Now let's train an XGBoost model! For this question, we'll tune the `eta` parameter

* Install XGBoost
* Create DMatrix for train and validation
* Create a watchlist
* Train a model with these parameters for 100 rounds:

```
xgb_params = {
    'eta': 0.3, 
    'max_depth': 6,
    'min_child_weight': 1,
    
    'objective': 'reg:squarederror',
    'nthread': 8,
    
    'seed': 1,
    'verbosity': 1,
}
```

In [10]:
%%capture output
import xgboost as xgb

def train_gradient_boost_model(dmX, watchlist, eta=0.3):
    xgb_params = {
        'eta': eta, 
        'max_depth': 6,
        'min_child_weight': 1,
        'objective': 'reg:squarederror',
        'nthread': 8,
        'seed': 1,
        'verbosity': 1,
    }

    return xgb.train(xgb_params, dmX, evals=watchlist, verbose_eval=5, num_boost_round=100)

dm_trainX = xgb.DMatrix(trainX_encoded, label=trainy, feature_names=dv.feature_names_)
dm_valX = xgb.DMatrix(valX_encoded, label=valy, feature_names=dv.feature_names_)
watchlist = [(dm_trainX, 'train'), (dm_valX, 'validation')]

train_gradient_boost_model(dm_trainX, watchlist)

In [11]:
def parse_line(line):
    _num, train, val = line.split('\t')
    return (float(train.split(':')[1]), float(val.split(':')[1]))

def parse_xgb_output(output, eta):
    return pd.DataFrame([parse_line(line) for line in output.stdout.strip().split('\n')],
                        columns=[f'train_rmse;eta={eta}', f'val_rmse;eta={eta}'])

rmse_eta_03 = parse_xgb_output(output, 0.3)

Now change `eta` first to `0.1` and then to `0.01`

In [12]:
%%capture output
train_gradient_boost_model(dm_trainX, watchlist, eta=0.1)

In [13]:
rmse_eta_01 = parse_xgb_output(output, 0.1)

In [14]:
%%capture output
train_gradient_boost_model(dm_trainX, watchlist, eta=0.01)

In [15]:
rmse_eta_001 = parse_xgb_output(output, 0.01)

In [16]:
pd.DataFrame(np.c_[rmse_eta_03, rmse_eta_01, rmse_eta_001],
             columns=rmse_eta_03.columns.tolist() + rmse_eta_01.columns.tolist() + rmse_eta_001.columns.tolist())

,train_rmse;eta=0.3,val_rmse;eta=0.3,train_rmse;eta=0.1,val_rmse;eta=0.1,train_rmse;eta=0.01,val_rmse;eta=0.01
0,3.02752,3.02415,3.87217,3.86889,4.25336,4.25010
1,0.67490,0.67752,2.31905,2.31692,4.04779,4.04454
2,0.43912,0.44981,1.41910,1.41786,3.85242,3.84921
3,0.42259,0.43827,0.91299,0.91348,3.66674,3.66359
4,0.41716,0.43691,0.64528,0.64883,3.49030,3.48719
5,0.41365,0.43621,0.51733,0.52364,3.32263,3.31956
6,0.40712,0.43543,0.46186,0.47101,3.16332,3.16029
7,0.40444,0.43510,0.43843,0.44997,3.01196,3.00898
8,0.40103,0.43466,0.42770,0.44150,2.86817,2.86533
9,0.39723,0.43371,0.42222,0.43795,2.73158,2.72884


What's the best eta?

* 0.3
* 0.1
* 0.01

## Submit the results


Submit your results here: https://forms.gle/wQgFkYE6CtdDed4w8

It's possible that your answers won't match exactly. If it's the case, select the closest one.


## Deadline


The deadline for submitting is 20 October 2021, 17:00 CET (Wednesday). After that, the form will be closed.

